# Introduction
This notebook was created by Garrett R. Dowdy (gdowdy3@gmail.com) to solve HSHS's shift assignment problem.

The notebook consists of a series of gray boxes ("cells") containing code.
The cells are divided up into sections by bold headers  -- "Introduction", "Setup", and so on.
Many of these headers are accompanied by explanatory text.

To run this code:

1. Verify that the "Individual Preferences.csv" and "Prefilled Shifts.csv" files have been uploaded to the same JuliaBox folder as the .pynb file that you clicked to open this notebook. Note: the spelling must be *exact*.
2. On the menu bar above, select **Cell** --> **All Output** --> **Clear**
3. Moving down the document, run each cell in sequence,  reading the explanatory text and checking for errors as you go.
4. Check the summary statistics printed by the final cell.
5. Check the output csv files added to the current JuliaBox folder (see previous tab in your browser).
6. If you are happy with the results, great!  If not, tweak the objective weights in the "Define the Objective Function" section and start over, returning to Step 2.
7. You can exit this notebook gracefully by selecting **File** --> **Close and Halt**.

A cell is run by clicking in the gray box and then pressing [ctrl] + [enter] on your keyboard. 
You will know that a cell is executed when the "In[ ]" to its left is replaced with "In[#]", where "#" is a number. 
"In[\*]" indicates that the cell is currently executing.  You can use the up and down arrow keys to move between cells.

# Setup

## Define Custom Functions
The code makes use of some custom-made functions.  So that they don't disrupt the flow of logic in the body of the code below, it is best to define them here.

In [ ]:
function mapShiftStringToShiftInd(shiftString::String)
#Inputs:
#   shiftString = a string describing a shift. E.g., "Monday Dinner" or
#   "Saturday Evening".
#
#Outputs:
#   shiftInd = the corresponding shift index, based on the following table:
#           Shift String        Shift Index
#           Monday Breakfast    1
#           Monday Dinner       2
#           Monday Evening      3
#           Monday Overnight    4
#           Tuesday Breakfast   5
#           .
#           .
#           .
#           Sunday Evening      27
#           Sunday Overnight    28

    #translate the string into a shift index
    spacePos = searchindex(shiftString," ");
    day = shiftString[1:(spacePos-1)];
    period = shiftString[(spacePos+1):end];

    #initialize the day dictionary
    dayDict = Dict{String,Int}()

    #add the entries to the day dictionary
    dayDict["Monday"] = 0
    dayDict["Tuesday"] = 1
    dayDict["Wednesday"] = 2
    dayDict["Thursday"] = 3
    dayDict["Friday"] = 4
    dayDict["Saturday"] = 5
    dayDict["Sunday"] = 6

    #retrieve the index corresponding to the day
    s_d = dayDict[day]

    #initialize the period dictionary
    periodDict = Dict{String,Int}()

    #add the entries to the period dictionary
    periodDict["Breakfast"] = 1
    periodDict["Dinner"] = 2
    periodDict["Evening"] = 3
    periodDict["Overnight"] = 4

    #retrieve the index corresponding to the period
    s_p = periodDict[period]

    #calculate the shift index
    shiftInd = 4*s_d + s_p;

    return shiftInd
end

In [ ]:
function mapShiftStringToVolunteersRequired(shiftString::String)
#Inputs:
#   shiftString = a string describing a shift. E.g., 'Monday Dinner' or
#   'Saturday Evening'.
#
#Outputs:
#   n = the number of volunteers required for the specified shift,
#   according to the following table:
#           Shift Period        Volunteers Required
#           Breakfast           6
#           Dinner              5
#           Evening             4
#           Overnight           4


    #translate the string into a shift index
    spacePos = searchindex(shiftString," ");
    period = shiftString[(spacePos + 1):end];

    if period == "Breakfast"
        n = 6
    elseif period == "Dinner"
        n = 5
    elseif period == "Evening"
        n = 4
    elseif period == "Overnight"
        n = 4
    else
        error("The given shift string $shiftString does not match the expected format.")    
    end

    return n

end

In [ ]:
function mapShiftIndToShiftString(s)
#Inputs:
#   s = an index, which has a corresponding shift.  The relationship
#       is established by the following table:
#           Shift String        Shift Index
#           Monday Breakfast    1
#           Monday Dinner       2
#           Monday Evening      3
#           Monday Overnight    4
#           Tuesday Breakfast   5
#           .
#           .
#           .
#           Sunday Evening      27
#           Sunday Overnight    28
#
#Outputs:
#   shiftString = the shift string taken from the above table.


    #figure out which day this corresponds to
    dayInd = ceil(s/4);
    if dayInd == 1
        dayStr = "Monday"
    elseif dayInd == 2
        dayStr = "Tuesday"
    elseif dayInd == 3
        dayStr = "Wednesday"
    elseif dayInd == 4
        dayStr = "Thursday"
    elseif dayInd == 5
        dayStr = "Friday"
    elseif dayInd == 6
        dayStr = "Saturday"
    elseif dayInd == 7
        dayStr = "Sunday"
    else
        error("The given shift index $s is outside of the expected range.")
    end

    # figure out which period this corresponds to
    perInd = s%4
    if perInd == 0
        perStr = "Overnight"
    elseif perInd == 1
        perStr = "Breakfast"
    elseif perInd == 2
        perStr = "Dinner"
    elseif perInd == 3
        perStr = "Evening"
    else
        error("The given shift index $s is outside of the expected range.")
    end

    # construct the shift string
    shiftString = "$dayStr $perStr";

    return shiftString
end

# Read in the Problem Data
There are two sources of data which define the problem:

1. A csv file showing the shift assignment preferences for each individual: "Individual Preferences.csv".
2. A csv file showing which shifts have been prefilled: "Prefilled Shifts.csv".

In this section of the code, these two data files are imported and interpreted.

Executing the following two cells displays the top and bottom portions of the preferences input table.  Check the output to make sure there isn't any funny business going on.

In [ ]:
# Read in the individual preferences
using DataFrames, DataArrays
raw = readtable("Individual Preferences.csv")

In [ ]:
DataFrames.tail(raw)

In [ ]:
# describe the input table
prefVolunteersCol = 1;
firstNameCol = 2;
lastNameCol = 3;
firstPrefCol = 4;

# count the number of volunteers
n_V = size(raw,1);

# build the list of preferred volunteers
V_star = Int64[]
for v = 1:n_V
    if typeof(raw[v,prefVolunteersCol]) == Bool && raw[v,prefVolunteersCol] == true
        push!(V_star, v);
    end
end

# extract the volunteers' names
volNames = Array{String,1}(n_V);
for v = 1:n_V
    firstName = raw[v,firstNameCol]
    lastName = raw[v,lastNameCol]
    volNames[v] = "$firstName $lastName"
end

# read in the volunteer preferences
n_S = 4*7;  #shifts per day times number of days = total number of shifts

## initialize the preference matrix 
p = Array{Int}(n_V,n_S)
fill!(p,0)

volPrefs = raw[1:n_V,firstPrefCol:(firstPrefCol+4)];
for v = 1:n_V
    for j = 1:5
        # extract the jth choice of volunteer v
        prefShift = volPrefs[v,j];
        
        # check for a string
        if typeof(prefShift) == String #this is a string
            # translate the string into a shift index
            s = mapShiftStringToShiftInd(prefShift);
            
            # assign a value to the appropriate entry of the preference matrix
            p[v,s] = 6 - j; #implies that "first choice" gets 5 priority points.
            
        else #not a string
            #do nothing
        end
        
    end
end

Similarly, executing the following two cells displays the top and bottom portions of the prefilled shifts input table.  Check the output to make sure there isn't any funny business going on.

In [ ]:
# read in the pre-filled shifts
raw = readtable("Prefilled Shifts.csv")

In [ ]:
DataFrames.tail(raw)

In [ ]:
# initialize the set of pre-filled shift (blocks)
n_S_T = size(raw,1)
S_T = Array{Int}(n_S_T)

# initialize the count of pre-filled shifts;
prefilled_shifts = 0;

# initialize the array of group names
groupNames = Array{String}(n_S_T)

# describe the table
headRow = 1;
shiftCol = 1;
groupCol = 2;

# loop over the pre-filled shifts
for j = 1:n_S_T
    # extract the shift string
    shiftString = raw[j,shiftCol];
    
    # extract the group name string
    groupNames[j] = raw[j,groupCol];
    
    # convert it into an index
    s = mapShiftStringToShiftInd(shiftString);
    
    # increase the count of pre-filled shifts
    n = mapShiftStringToVolunteersRequired(shiftString);
    prefilled_shifts += n;
    
    # add this index to the set of pre-filled shifts
    S_T[j] = s
end

# Define the other sets
V = 1:n_V;
S = 1:n_S;
S_B = 4*(0:6) + 1;
S_D = 4*(0:6) + 2;
S_E = 4*(0:6) + 3;
S_O = 4*(0:6) + 4;

# Define and Solve the Optimization Problem
## Initialize the Model and Define the Decision Variables

In [ ]:
# Describe the Optimization Problem
using JuMP
using Cbc

# m = Model()  #original line
m = Model(solver=CbcSolver())

## define the decision variables
@variable(m,x[1:n_V,1:n_S],Bin)

## Define the Constraints

In [ ]:
## define the constraints
### constraint 1
for v in V
    @constraint(m,sum(x[v,:]) <= 1)
end

### constraint 2
@constraint(m,x .<= p)

### constraint 3
for s in S_D
    @constraint(m,sum(x[:,s]) <= 5)
end

### constraint 4
for s in S_E
    @constraint(m,sum(x[:,s]) <= 4)
end

### constraint 5
for s in S_O
    @constraint(m,sum(x[:,s]) <= 4)
end

### constraint 6
for s in S_B
    @constraint(m,sum(x[:,s]) <= 6)
end

### constraint 7
for s in S_T
    @constraint(m,x[:,s] .== 0)
end

## Define the Objective Function
The objective function has several weighting coefficients, $w_1, w_2, w_3$, and $w_4$.
Each of these coefficients corresponds to a particular objective.

| Weighting Coefficient | Suggested Value | Objective |
|:---------------------:|:---------------:|-----------|
| $w_1$                 | 1               | Assignments are made based on preferences.      |
| $w_2$                 | 5               |The preferred volunteers are more likely to be assigned a shift. |
| $w_3$                 | 1               |Fill the most-difficult-to-fill shifts first. In particular, the overnight shift is typically the hardest to fill, the dinner and evening shifts are semi-hard to fill, and breakfast is easy. |
| $w_4$                 | 1               |All shifts need to be covered.  |

You can set the values of these coefficients in the code below. 
Each coefficient most be nonnegative (i.e., $w_i \geq 0$ for $i \in \{1,...,4\}$).
**The greater the value of the coefficient, the harder the code will try to accomplish the corresponding objective.**
That being said, if you double each coefficient, it will make no difference to the code, because it is really the *ratios* between them that matter.

Manipulating these weighting coefficients will very likely effect the summary statistics and schedule produced by the code, but small changes in the weights may have no effect.

In [ ]:
## define the objective function
### define the objective weights
w1 = 1
w2 = 5
w3 = 1
w4 = 1

### objective 1
c1 = p

### objective 2
c2 = zeros(n_V,n_S)
for v in V_star
    c2[v,:] = 1
end

### objective 3
c3 = zeros(n_V,n_S)
c3[:,S_O] = 2
c3[:,S_D] = 1
c3[:,S_E] = 1
c3[:,S_B] = 0

### objective 4
c4 = ones(n_V,n_S)

### add all the objectives together
c = w1*c1 + w2*c2 + w3*c3 + w4*c4

### define the objective function

@objective(m,Max,sum(sum(c.*x)));

## Solve the Optimization Problem
This cell should think for a few seconds, and then display one word: "Optimal".

In [ ]:
# Solve the Optimization Problem
status = solve(m)
println(status)

# extract the numerical solution
x = round(Int,getvalue(x));


# Export the Results
## Export Volunteer-Focused Table
This cell constructs and outputs a table listing all of the volunteers in the first column and their shift assignment (if any) in the second.

In [ ]:
# Export the results
## Export a table showing the assignment of each volunteer
### convert the numerical solution into plain english
shiftStrings = Array{String}(n_V)
for v in V
    # look to see if this volunteer was assigned
    if sum(x[v,:]) == 1 #this volunteer was assigned
        # extract the shift index
        s = findfirst(x[v,:]);
        
        # figure out which shift this corresponds to (in English)
        shiftStr = mapShiftIndToShiftString(s);
        
        # store the shift string
        shiftStrings[v] = shiftStr
    else
        shiftStrings[v] = ""
    end
end

# write the results to Excel
using DataArrays, DataFrames, CSV
volunteerAssignments = DataFrame(Volunteer = volNames, Shift_Assignment = shiftStrings)
CSV.write("Volunteer Focused Output.csv", volunteerAssignments);

## Export Shift-Focused Table
This cell constructs and outputs a table showing each shift in the first column and the people who have been assigned to the shift in subsequent columns. 
This table is preferred for checking that each shift has been covered.
Shifts that have not been covered will be marked with a warning.

In [ ]:
# Export a table showing the staffing of each shift
## initialize the cell arrays
shiftNames = fill("",n_S) #Array{String}(n_S);
shiftStaff = fill("",(n_S,6)) #Array{String}(n_S,6);
underStaffedWarning = fill("",n_S) #Array{String}(n_S);

## initialize the count of under-staffed shifts
underStaffedShifts = 0;

## loop over the shifts
for s in S
    # store the shift name
    shiftNames[s] = mapShiftIndToShiftString(s)
    
    # check whether or not this is a pre-filled shift
    if !(s in S_T) # this is not a prefilled shift
        # extract the names of the volunteers assigned to the shift
        tempShiftStaffNames = volNames[find(x[:,s])]
        for v = 1:length(tempShiftStaffNames)
            shiftStaff[s,v] = tempShiftStaffNames[v]
        end
        
        # check for understaffing
        if length(tempShiftStaffNames) < mapShiftStringToVolunteersRequired(mapShiftIndToShiftString(s))
            underStaffedWarning[s] = "WARNING: Under-staffed shift."
            
            # increment the count of understaffed shifts
            underStaffedShifts += 1
        end
    else # this is a prefilled shift
        shiftStaff[s,1] = groupNames[findfirst(S_T,s)];
    end
end

# put the table together
df = DataFrame(Shift = shiftNames, Shift_Staff1 = shiftStaff[:,1],Shift_Staff2 = shiftStaff[:,2],Shift_Staff3 = shiftStaff[:,3],Shift_Staff4 = shiftStaff[:,4],Shift_Staff5 = shiftStaff[:,5],Shift_Staff6 = shiftStaff[:,6], Under_Staffing_Warning = underStaffedWarning)

# write the results to Excel
using CSV
CSV.write("Shift Focused Output.csv",df);

# Display Statistics
This code prints some statistics that give you a sense of the quality of the schedule generated.

In [ ]:
# display some statistics
println("Fraction of volunteers assigned:")
n_assignments = sum(sum(x))
println(n_assignments/n_V)

println("Fraction of staffing requirements covered:")
println((n_assignments + prefilled_shifts)/(7*(5 + 4 + 4 + 6)))

println("Number of Under-Staffed Shifts:")
println(underStaffedShifts)

choiceSum = zeros(5,1)
for v in V
    s = findfirst(x[v,:])
    if s > 0
        choiceIndex = 6 - p[v,s]
        choiceSum[choiceIndex] = choiceSum[choiceIndex] + 1;
    end
end
choiceFraction = choiceSum ./ n_assignments;
println("Fraction of 1st choice assignments:")
println(choiceFraction[1]);
println("Fraction of 2nd choice assignments:")
println(choiceFraction[2]);
println("Fraction of 3rd choice assignments:")
println(choiceFraction[3]);
println("Fraction of 4th choice assignments:")
println(choiceFraction[4]);
println("Fraction of 5th choice assignments:")
println(choiceFraction[5]);
println("Sum of fractions:")
println(sum(choiceFraction));

println("Fraction of preferred volunteers assigned")
n_assignments = sum(sum(x[V_star,:]));
println(n_assignments/length(V_star))

# The End
If you like what you see, go to the JuliBox tab, which you still have open and check out the output files.
Instructions for doing this can be found in the README file in the GitHub repository for this code.

If you don't like what you see, consider tweaking the weights in the "Defining the Objective Function" section.
